### **NLTK**

In [17]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from string import punctuation
from heapq import nlargest


nltk.download('stopwords')
nltk.download('punkt')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [18]:
# Текст для обробки
text = """The Orbiter Discovery, OV-103, is considered eligible for listing in the National Register of Historic Places (NRHP) in the context of the U.S. Space Shuttle Program (1969-2011) under Criterion A in the areas of Space Exploration and Transportation and under Criterion C in the area of Engineering. Because it has achieved significance within the past fifty years, Criteria Consideration G applies. Under Criterion A, Discovery is significant as the oldest of the three extant orbiter vehicles constructed for the Space Shuttle Program (SSP), the longest running American space program to date; she was the third of five orbiters built by NASA. Unlike the Mercury, Gemini, and Apollo programs, the SSP’s emphasis was on cost effectiveness and reusability, and eventually the construction of a space station. Including her maiden voyage (launched August 30, 1984), Discovery flew to space thirty-nine times, more than any of the other four orbiters; she was also the first orbiter to fly twenty missions. She had the honor of being chosen as the Return to Flight vehicle after both the Challenger and Columbia accidents. Discovery was the first shuttle to fly with the redesigned SRBs, a result of the Challenger accident, and the first shuttle to fly with the Phase II and Block I SSME. Discovery also carried the Hubble Space Telescope to orbit and performed two of the five servicing missions to the observatory. She flew the first and last dedicated Department of Defense (DoD) missions, as well as the first unclassified defense-related mission. In addition, Discovery was vital to the construction of the International Space Station (ISS); she flew thirteen of the thirty-seven total missions flown to the station by a U.S. Space Shuttle. She was the first orbiter to dock to the ISS, and the first to perform an exchange of a resident crew. Under Criterion C, Discovery is significant as a feat of engineering. According to Wayne Hale, a flight director from Johnson Space Center, the Space Shuttle orbiter represents a “huge technological leap from expendable rockets and capsules to a reusable, winged, hypersonic, cargo-carrying spacecraft.” Although her base structure followed a conventional aircraft design, she used advanced materials that both minimized her weight for cargo-carrying purposes and featured low thermal expansion ratios, which provided a stable base for her Thermal Protection System (TPS) materials. The Space Shuttle orbiter also featured the first reusable TPS; all previous spaceflight vehicles had a single-use, ablative heat shield. Other notable engineering achievements of the orbiter included the first reusable orbital propulsion system, and the first two-fault-tolerant Integrated Avionics System. As Hale stated, the Space Shuttle remains “the largest, fastest, winged hypersonic aircraft in history,” having regularly flown at twenty-five times the speed of sound.
"""

In [19]:
# Токенізація
sentences = sent_tokenize(text)

word_frequencies = {}
for word in word_tokenize(text):
    if word.lower() not in stop_words and word.lower() not in punctuation:
        if word not in word_frequencies.keys():
            word_frequencies[word] = 1
        else:
            word_frequencies[word] += 1

In [20]:
maximum_frequency = max(word_frequencies.values())

# Нормалізація частот слів
for word in word_frequencies.keys():
    word_frequencies[word] = word_frequencies[word] / maximum_frequency

# Обчислення ваг пропозицій
sentence_scores = {}
for sentence in sentences:
    for word in word_tokenize(sentence.lower()):
        if word in word_frequencies.keys():
            if len(sentence.split(' ')) < 30:
                if sentence not in sentence_scores.keys():
                    sentence_scores[sentence] = word_frequencies[word]
                else:
                    sentence_scores[sentence] += word_frequencies[word]

In [21]:
# Вибираємо 30% пропозицій для резюмування
select_length = int(len(sentences) * 0.3)
summary = nlargest(select_length, sentence_scores, key=sentence_scores.get)
print(summary)

['Other notable engineering achievements of the orbiter included the first reusable orbital propulsion system, and the first two-fault-tolerant Integrated Avionics System.', 'The Space Shuttle orbiter also featured the first reusable TPS; all previous spaceflight vehicles had a single-use, ablative heat shield.', 'She flew the first and last dedicated Department of Defense (DoD) missions, as well as the first unclassified defense-related mission.', 'She was the first orbiter to dock to the ISS, and the first to perform an exchange of a resident crew.']


### **SpaCy**

In [22]:
import spacy


nlp = spacy.load('en_core_web_sm')

In [23]:
# Обробка тексту за допомогою SpaCy
doc = nlp(text)

# Виділення пропозицій
sentences = [sent.text for sent in doc.sents]

# Токенізація слів та видалення стоп-слів та розділових знаків
word_frequencies = {}
for word in doc:
    if word.text.lower() not in stop_words and word.text.lower() not in punctuation:
        if word.text not in word_frequencies.keys():
            word_frequencies[word.text] = 1
        else:
            word_frequencies[word.text] += 1

In [24]:
maximum_frequency = max(word_frequencies.values())

# Нормалізація частот слів
for word in word_frequencies.keys():
    word_frequencies[word] = word_frequencies[word] / maximum_frequency

# Обчислення ваг пропозицій
sentence_scores = {}
for sentence in sentences:
    for word in nlp(sentence.lower()):
        if word.text in word_frequencies.keys():
            if len(sentence.split(' ')) < 30:
                if sentence not in sentence_scores.keys():
                    sentence_scores[sentence] = word_frequencies[word.text]
                else:
                    sentence_scores[sentence] += word_frequencies[word.text]

In [25]:
# Вибираємо 30% пропозицій для резюмування
select_length = int(len(sentences) * 0.3)
summary = nlargest(select_length, sentence_scores, key=sentence_scores.get)
print(summary)

['Other notable engineering achievements of the orbiter included the first reusable orbital propulsion system, and the first two-fault-tolerant Integrated Avionics System.', 'The Space Shuttle orbiter also featured the first reusable TPS; all previous spaceflight vehicles had a single-use, ablative heat shield.', 'She flew the first and last dedicated Department of Defense (DoD) missions, as well as the first unclassified defense-related mission.', 'As Hale stated, the Space Shuttle remains “the largest, fastest, winged hypersonic aircraft in history,” having regularly flown at twenty-five times the speed of sound.\n', 'She was the first orbiter to dock to the ISS, and the first to perform an exchange of a resident crew.']
